In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/covid-semi-pred/covid-semi_pred.csv
/kaggle/input/d/ishangarg12/irdata/dataset.xlsx


In [2]:
!pip install simpletransformers transformers sacremoses

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.7/250.7 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 44.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 89.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 90.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 8.5 MB/s eta 0:00:00
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=af1fcfd42475dc8a5700a15b91f4fc65f357c1faa30615d131a23dc2ba59b2d7
  Stored in directory: /root/.cache/pip/wheels/00/24/97/a2ea5324f36bc626e1ea0267f33db6aa80d157ee977e9e42fb
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16181 sha256=fc4bf20cfb05e7e8e425b2c

In [3]:
# df = pd.read_excel("/kaggle/input/d/ishangarg12/irdata/dataset.xlsx")
df = pd.read_csv("data/corpus/covid-semi_pred.csv")
df.head()

,Concatenated,Contextual
0,Assessing the impacts of COVID-19 vaccination ...,1
1,Association between interleukin-10 gene polymo...,1
2,Quality of Life of early-stage breast-cancer p...,1
3,"The research interest, capacity and culture of...",0
4,Machine learning prediction for COVID-19 disea...,1


In [4]:
# df.dropna(subset=["Contextual", "Article title", "Article abstract"], inplace=True)

In [5]:
def concatenate(title, keyword, abstract):
    title = f"{title}"
    if type(keyword) == float:
        # print("Keyword: ", keyword)
        keyword = ""
    else:
        keyword = f"{keyword}."
    abstract = f"{abstract}"
    
    return title + "$" + keyword + "$" + abstract

In [6]:
# df["Concatenated"] = df.apply(
#         lambda x: concatenate(x["Article title"], x["Article keywords"], x["Article abstract"]), axis=1,
#     )

In [7]:
# df = df.astype({"Contextual": "int32"})
# df = df[["Concatenated", "Contextual"]]
# df.reset_index(drop=True, inplace=True)

In [8]:
# df.head()

In [9]:
true_pred_split = 188
train_df = df[0:true_pred_split]
test_df = df[true_pred_split:]
len(train_df), len(test_df)

(188, 8645)

In [10]:
from sklearn.model_selection import train_test_split
train_df, eval_df = train_test_split(train_df, test_size=0.2, random_state=84)
train_df = pd.concat([train_df, test_df])
train_df.reset_index(drop=True, inplace=True)
train_df = pd.DataFrame(
        {
            "text": train_df["Concatenated"].replace(r"\n", " ", regex=True),
            "label": train_df["Contextual"],
        },
        columns=["text", "label"],
    )

train_df.reset_index(drop=True, inplace=True)

    # print(train_df.head())

eval_df = pd.DataFrame(
        {
            "text": eval_df["Concatenated"].replace(r"\n", " ", regex=True),
            "label": eval_df["Contextual"],
        },
        columns=["text", "label"],
    )

eval_df.reset_index(drop=True, inplace=True)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [11]:
len(train_df), len(eval_df)

(8795, 38)

In [12]:
from simpletransformers.classification import ClassificationModel

In [13]:
# import shutil
# shutil.rmtree("/kaggle/working/outputs")
# shutil.rmtree("/kaggle/working/runs")
# shutil.rmtree("/kaggle/working/cache_dir")

In [14]:
args = {
    'no_cache': True,
    'no_save': True,
    'num_train_epochs': 8,
    'use_early_stopping': True,
    'evaluate_during_training': False,
    'overwrite_output_dir': True
}

In [15]:
model = ClassificationModel('roberta','roberta-base', num_labels=2, use_cuda=True, args=args)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifi

In [16]:
# Train the model
model.train_model(train_df, eval_df = eval_df)

/opt/conda/lib/python3.10/site-packages/simpletransformers/classification/classification_model.py:612: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/8795 [00:00<?, ?it/s]

Epoch:   0%|          | 0/8 [00:00<?, ?it/s]

Running Epoch 0 of 8:   0%|          | 0/1100 [00:00<?, ?it/s]

Running Epoch 1 of 8:   0%|          | 0/1100 [00:00<?, ?it/s]

...

Running Epoch 4 of 8:   0%|          | 0/1100 [00:00<?, ?it/s]

Running Epoch 5 of 8:   0%|          | 0/1100 [00:00<?, ?it/s]

Running Epoch 6 of 8:   0%|          | 0/1100 [00:00<?, ?it/s]

Running Epoch 7 of 8:   0%|          | 0/1100 [00:00<?, ?it/s]

(8800, 0.6915105991742828)

In [17]:


# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(eval_df)

print(result)

accuracy = (result["tp"] + result["tn"]) / (result["tp"] + result["tn"] + result["fp"] + result["fn"])

precision = result['tp']/(result['tp'] + result['fp'])

recall = result['tp']/(result['tp'] + result['fn'])

f1 = (precision*recall)/(precision+recall)

print("Accuracy: ", accuracy)
print('Precision: ', precision)
print('Recall: ', recall)
print('F1 Score: ', f1)

/opt/conda/lib/python3.10/site-packages/simpletransformers/classification/classification_model.py:1454: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/38 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/5 [00:00<?, ?it/s]

{'mcc': 0.0, 'tp': 22, 'tn': 0, 'fp': 16, 'fn': 0, 'auroc': 0.7386363636363636, 'auprc': 0.7900401201648769, 'eval_loss': 0.683544921875}
Accuracy:  0.5789473684210527
Precision:  0.5789473684210527
Recall:  1.0
F1 Score:  0.3666666666666667
